In [30]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, early_stopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

# Configuration
CSV_URL = "https://docs.google.com/spreadsheets/d/1fZhDRbXM96enJFXnsP8eRJEVOcQ5agxaapMI9GfW_PQ/export?format=csv"
LAG_CONFIGS = [2, 3, 4, 5, 6, 7, 8, 9, 10]
MIN_TRAIN_YEARS = 3
RANDOM_STATE = 42
THRESHOLD_PROB = 0.5
PLOT_DIR = "plots"
TOP_N_FEATURES = 10

# Model hyperparameters
N_ESTIMATORS_VALIDATION = 200
N_ESTIMATORS_FINAL = 300
EARLY_STOPPING_ROUNDS = 30

# Feature engineering options
INCLUDE_ALTERNATE = True
STRIDE = 2

# Warning thresholds
F1_VARIANCE_THRESHOLD = 0.15  # Alert if F1 range exceeds this
UNIQUE_PROB_RATIO = 0.5  # Alert if unique probabilities < this ratio

os.makedirs(PLOT_DIR, exist_ok=True)


def load_and_prepare_data(url):
    """Load data from CSV and convert to long format with binary appear flag."""
    print("Loading data...")

    try:
        df_wide = pd.read_csv(url)
    except Exception as e:
        raise ValueError(f"Failed to load CSV from {url}: {str(e)}")

    if df_wide.empty:
        raise ValueError("CSV loaded but contains no data")

    df_wide = df_wide.rename(columns={df_wide.columns[0]: 'topic'})

    if 'topic' not in df_wide.columns:
        raise ValueError("CSV must have a 'topic' column")

    # Extract year columns (excluding 'deferred' columns)
    year_cols = [col for col in df_wide.columns
                 if col != 'topic' and 'deferred' not in str(col).lower()]

    if len(year_cols) == 0:
        raise ValueError("No year columns found in CSV")

    df_wide = df_wide[['topic'] + year_cols]

    # Create year mapping - try clean integers first, fallback to regex
    year_map = {}
    for col in year_cols:
        # Try direct integer conversion for clean year columns like "2015"
        if col.isdigit() and len(col) == 4:
            year_map[col] = int(col)
        else:
            # Fallback to regex for formatted columns like "Year_2015"
            match = re.search(r'\d{4}', str(col))
            if match:
                year_map[col] = int(match.group())

    if len(year_map) == 0:
        raise ValueError("Could not extract year information from columns")

    # Convert to binary appearance
    def appeared(cell):
        return 0 if pd.isna(cell) or str(cell).strip() == '' else 1

    for col in year_cols:
        df_wide[col] = df_wide[col].apply(appeared)

    # Melt to long format
    df = pd.melt(df_wide, id_vars=['topic'], value_vars=year_cols,
                 var_name='col', value_name='appear')
    df['year'] = df['col'].map(year_map)
    df = df[['year', 'topic', 'appear']].dropna().sort_values(['topic', 'year'])
    df['year'] = df['year'].astype(int)

    # Fill missing year-topic combinations with 0
    all_years = np.arange(df['year'].min(), df['year'].max() + 1)
    topics = df['topic'].unique()
    index = pd.MultiIndex.from_product([topics, all_years], names=['topic', 'year'])
    df = df.set_index(['topic', 'year']).reindex(index, fill_value=0).reset_index()
    df['appear'] = df['appear'].astype(int)

    return df


def encode_topics(df):
    """Create topic_id encoding once for consistency."""
    le = LabelEncoder()
    df['topic_id'] = le.fit_transform(df['topic'])
    return df, le


def build_lag_features(years, values, target_year, n_lags, include_alternate=False, stride=2):
    """
    Pure function to build lag features for a single target year.

    Args:
        years: Array of historical years
        values: Array of historical appearance values (0 or 1)
        target_year: Year to predict
        n_lags: Number of lag features
        include_alternate: Whether to include strided lags
        stride: Stride for alternate lags

    Returns:
        Dict of lag features
    """
    year_to_val = dict(zip(years, values))
    features = {}

    # Consecutive lags: year-1, year-2, ..., year-n
    lag_vals = []
    for lag in range(1, n_lags + 1):
        val = year_to_val.get(target_year - lag, 0)
        features[f'lag_{lag}'] = val
        lag_vals.append(val)

    features['lag_mean'] = np.mean(lag_vals)

    # Alternate (strided) lags: year-stride, year-2*stride, ..., year-n*stride
    if include_alternate:
        alt_vals = []
        for lag in range(1, n_lags + 1):
            val = year_to_val.get(target_year - lag * stride, 0)
            features[f'alt_lag_{lag}'] = val
            alt_vals.append(val)

        features['alt_lag_mean'] = np.mean(alt_vals)

    return features


def create_features(df, n_lags, include_alternate=False, stride=2):
    """
    Create lag features for all rows in the dataset.

    Args:
        df: DataFrame with 'topic', 'year', 'appear', 'topic_id' columns
        n_lags: Number of lag features to create
        include_alternate: Whether to include strided lag features
        stride: Stride for alternate lags

    Returns:
        DataFrame with lag features, list of feature column names
    """
    df = df.copy()
    rows = []

    for topic in df['topic'].unique():
        topic_data = df[df['topic'] == topic].sort_values('year')
        years = topic_data['year'].values
        values = topic_data['appear'].values
        topic_id = topic_data['topic_id'].iloc[0]

        for i, target_year in enumerate(years):
            # Build lag features for this year
            lag_dict = build_lag_features(
                years[:i+1],  # Only use history up to this point
                values[:i+1],
                target_year,
                n_lags,
                include_alternate,
                stride
            )

            row = {
                'topic': topic,
                'year': target_year,
                'appear': values[i],
                'topic_id': topic_id,
                **lag_dict
            }
            rows.append(row)

    df_with_lags = pd.DataFrame(rows)

    # Define feature set
    lag_cols = [f'lag_{i}' for i in range(1, n_lags + 1)]
    features = lag_cols + ['lag_mean', 'year', 'topic_id']

    if include_alternate:
        alt_cols = [f'alt_lag_{i}' for i in range(1, n_lags + 1)]
        features = lag_cols + alt_cols + ['lag_mean', 'alt_lag_mean', 'year', 'topic_id']

    return df_with_lags, features


def validate_configuration(df, n_lags, features, min_train_years):
    """
    Validate a single lag configuration using time-series cross-validation.

    Returns:
        List of dicts with validation results for each test year
    """
    min_year = df['year'].min()
    available_years = sorted(df['year'].unique())

    earliest_test_year = min_year + n_lags + min_train_years
    test_years = [y for y in available_years if y >= earliest_test_year]

    if len(test_years) == 0:
        return []

    X = df[features]
    y = df['appear']
    year_scores = []

    for test_year in test_years:
        train_idx = df['year'] < test_year
        test_idx = df['year'] == test_year

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        train_years_available = df[train_idx]['year'].nunique()

        if len(X_test) == 0 or len(X_train) == 0:
            continue
        if train_years_available < min_train_years:
            continue

        model = LGBMClassifier(
            n_estimators=N_ESTIMATORS_VALIDATION,
            random_state=RANDOM_STATE,
            verbose=-1
        )
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        year_scores.append({
            'year': test_year,
            'accuracy': acc,
            'f1': f1,
            'train_years': train_years_available
        })
        print(f"  {test_year}: Acc={acc:.3f}, F1={f1:.3f} "
              f"(trained on {train_years_available} years)")

    return year_scores


def analyze_year_patterns(df, year_scores):
    """
    Analyze why certain years have different F1 scores.
    Helps diagnose overfitting vs. genuinely easier/harder years.
    """
    print("\n  Year-by-year analysis:")
    for score in year_scores:
        year = score['year']
        year_data = df[df['year'] == year]
        appearance_rate = year_data['appear'].mean()

        # Calculate repeat rate (appeared in previous year)
        if year > df['year'].min():
            prev_year_data = df[df['year'] == year - 1]
            current_topics = set(year_data[year_data['appear'] == 1]['topic'])
            prev_topics = set(prev_year_data[prev_year_data['appear'] == 1]['topic'])
            repeat_rate = len(current_topics & prev_topics) / len(current_topics) if current_topics else 0
        else:
            repeat_rate = 0

        print(f"    {year}: F1={score['f1']:.3f}, "
              f"appear_rate={appearance_rate:.2f}, "
              f"repeat_rate={repeat_rate:.2f}")


def run_validation(df, lag_configs, include_alternate, stride):
    """Run validation across all lag configurations."""
    print("\n" + "="*80)
    print(f"VALIDATION: Testing {len(lag_configs)} configurations")
    print(f"Alternate lags: {'enabled' if include_alternate else 'disabled'} "
          f"(stride={stride if include_alternate else 'N/A'})")
    print("="*80)

    all_results = []

    for n_lags in lag_configs:
        print(f"\nTesting {n_lags}-year lookback")

        df_test, features = create_features(
            df.copy(), n_lags,
            include_alternate=include_alternate,
            stride=stride
        )

        year_scores = validate_configuration(
            df_test, n_lags, features, MIN_TRAIN_YEARS
        )

        if len(year_scores) == 0:
            print("  Skipped - insufficient data")
            continue

        avg_acc = np.mean([s['accuracy'] for s in year_scores])
        avg_f1 = np.mean([s['f1'] for s in year_scores])
        std_f1 = np.std([s['f1'] for s in year_scores])
        min_f1 = min([s['f1'] for s in year_scores])
        max_f1 = max([s['f1'] for s in year_scores])

        print(f"  Average: Acc={avg_acc:.3f}, F1={avg_f1:.3f} (±{std_f1:.3f})")
        print(f"  Range: F1 min={min_f1:.3f}, max={max_f1:.3f}")
        print(f"  Validated on {len(year_scores)} years")

        all_results.append({
            'n_lags': n_lags,
            'avg_acc': avg_acc,
            'avg_f1': avg_f1,
            'std_f1': std_f1,
            'min_f1': min_f1,
            'max_f1': max_f1,
            'n_years_tested': len(year_scores),
            'year_scores': year_scores
        })

    return all_results


def train_final_model(df, best_config, include_alternate, stride):
    """Train final model on all available data."""
    print(f"\nTraining final model with {best_config}-year lookback")

    df_final, features = create_features(
        df.copy(), best_config,
        include_alternate=include_alternate,
        stride=stride
    )

    X = df_final[features]
    y = df_final['appear']

    last_year = df_final['year'].max()
    train_idx = df_final['year'] < last_year
    test_idx = df_final['year'] == last_year

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = LGBMClassifier(
        n_estimators=N_ESTIMATORS_FINAL,
        random_state=RANDOM_STATE,
        verbose=-1
    )

    if len(X_test) > 0:
        model.fit(
            X_train, y_train,
            eval_set=[(X_test, y_test)],
            callbacks=[early_stopping(EARLY_STOPPING_ROUNDS)]
        )
        pred_test = model.predict(X_test)
        print(f"\nFinal model performance on {last_year}:")
        print(f"  Accuracy: {accuracy_score(y_test, pred_test):.3f}")
        print(f"  F1 Score: {f1_score(y_test, pred_test, zero_division=0):.3f}")
    else:
        model.fit(X_train, y_train)
        print("\nNo holdout test set available")

    # Feature importance
    fi = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    print(f"\nTop {TOP_N_FEATURES} most important features:")
    print(fi.head(TOP_N_FEATURES).to_string(index=False))

    return model, df_final, features


def predict_next_year(df, best_config, include_alternate, stride, le):
    """Generate predictions for next year using the pure function approach."""
    last_year = df['year'].max()
    next_year = last_year + 1

    predictions = []

    for topic in df['topic'].unique():
        topic_data = df[df['topic'] == topic].sort_values('year')
        years = topic_data['year'].values
        values = topic_data['appear'].values
        topic_id = topic_data['topic_id'].iloc[0]

        # Build lag features for next year
        lag_dict = build_lag_features(
            years, values, next_year, best_config,
            include_alternate=include_alternate,
            stride=stride
        )

        row = {
            'topic': topic,
            'year': next_year,
            'topic_id': topic_id,
            **lag_dict
        }
        predictions.append(row)

    return pd.DataFrame(predictions), next_year


def save_results(results, all_results, next_year, include_alternate, stride):
    """Save predictions and validation results to CSV."""
    # Save predictions
    alt_str = f"alt{int(include_alternate)}_s{stride}"
    filename = f"LC_CHEMISTRY_{next_year}_PREDICTIONS_{alt_str}.csv"
    results.to_csv(filename, index=False)
    print(f"\nPredictions saved to {filename}")

    # Save detailed validation
    val_records = []
    for r in all_results:
        for score in r['year_scores']:
            val_records.append({
                'n_lags': r['n_lags'],
                'test_year': score['year'],
                'train_years_used': score['train_years'],
                'accuracy': score['accuracy'],
                'f1': score['f1']
            })

    val_df = pd.DataFrame(val_records)
    val_filename = f"validation_results_{alt_str}.csv"
    val_df.to_csv(val_filename, index=False)
    print(f"Validation results saved to {val_filename}")

    return val_df


def plot_validation_summary(all_results, best_config, include_alternate, stride):
    """Create compact validation summary plot."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    lags = [r['n_lags'] for r in all_results]
    f1s = [r['avg_f1'] for r in all_results]
    stds = [r['std_f1'] for r in all_results]
    mins = [r['min_f1'] for r in all_results]
    maxs = [r['max_f1'] for r in all_results]

    # Left: F1 scores with range
    ax1.plot(lags, f1s, marker='o', linewidth=2, markersize=8,
             label='Average F1', color='#2E86AB')
    ax1.fill_between(lags, mins, maxs, alpha=0.2, color='#2E86AB',
                      label='Min-Max Range')
    ax1.axvline(best_config, color='#A23B72', linestyle='--',
                linewidth=2, alpha=0.7, label=f'Best: {best_config} years')
    ax1.set_xlabel('Lookback Window (years)', fontsize=11)
    ax1.set_ylabel('F1 Score', fontsize=11)
    ax1.set_title('Performance vs Lookback Window', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    ax1.set_ylim([0, 1])

    # Right: Consistency
    ax2.bar(lags, stds, alpha=0.7, color='#F18F01', edgecolor='black', linewidth=1)
    ax2.set_xlabel('Lookback Window (years)', fontsize=11)
    ax2.set_ylabel('Standard Deviation of F1', fontsize=11)
    ax2.set_title('Model Consistency (Lower = Better)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')

    for lag, std in zip(lags, stds):
        ax2.text(lag, std, f'{std:.3f}', ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    alt_str = f"alt{int(include_alternate)}_s{stride}"
    filename = f'validation_summary_{alt_str}.png'
    plt.savefig(os.path.join(PLOT_DIR, filename), dpi=150, bbox_inches='tight')
    print(f"Validation plot saved to plots/{filename}")
    plt.close()


def main():
    # Load and prepare data
    df = load_and_prepare_data(CSV_URL)
    df, le = encode_topics(df)

    min_year = df['year'].min()
    max_year = df['year'].max()
    print(f"Data spans: {min_year} to {max_year} ({max_year - min_year + 1} years)")
    print(f"Topics tracked: {df['topic'].nunique()}")

    # Run validation
    all_results = run_validation(df, LAG_CONFIGS, INCLUDE_ALTERNATE, STRIDE)

    if len(all_results) == 0:
        print("\nERROR: No valid configurations found")
        return

    # Find best configuration
    best_config = max(all_results, key=lambda x: x['avg_f1'])
    best_n_lags = best_config['n_lags']

    print("\n" + "="*80)
    print(f"BEST CONFIGURATION: {best_n_lags}-year lookback")
    print(f"  Average F1: {best_config['avg_f1']:.4f}")
    print(f"  Consistency: ±{best_config['std_f1']:.4f}")
    print(f"  F1 range: [{best_config['min_f1']:.3f}, {best_config['max_f1']:.3f}]")
    print(f"  Tested on: {best_config['n_years_tested']} years")

    # Check for high variance and diagnose
    f1_range = best_config['max_f1'] - best_config['min_f1']
    if f1_range > F1_VARIANCE_THRESHOLD:
        print(f"\n  WARNING: Large F1 variance detected ({f1_range:.3f})")
        print("  Model performance varies significantly across years")
        print("  Investigating year-specific patterns...")
        analyze_year_patterns(df, best_config['year_scores'])

    print("="*80)

    # Train final model
    model, df_final, features = train_final_model(
        df, best_n_lags, INCLUDE_ALTERNATE, STRIDE
    )

    # Generate predictions using pure function approach
    pred_df, next_year = predict_next_year(
        df, best_n_lags, INCLUDE_ALTERNATE, STRIDE, le
    )

    # Get probabilities
    X_future = pred_df[features]
    probs = model.predict_proba(X_future)[:, 1]

    # Build results
    results = pd.DataFrame({
        'topic': pred_df['topic'].values,
        'prob_2026': probs,
        'WILL_APPEAR': (probs >= THRESHOLD_PROB).astype(int)
    })

    # Add lag features for reference
    for col in [c for c in pred_df.columns if c.startswith('lag_') or c.startswith('alt_lag_')]:
        results[col] = pred_df[col].values

    results = results.sort_values('prob_2026', ascending=False).reset_index(drop=True)
    results['Rank'] = results.index + 1

    # Display predictions
    print("\n" + "="*80)
    print(f"{next_year} PREDICTIONS")
    print("="*80)
    print(results[['Rank', 'topic', 'prob_2026', 'WILL_APPEAR']].head(20).to_string(index=False))

    # Check for suspiciously identical probabilities
    unique_probs = results['prob_2026'].nunique()
    if unique_probs < len(results) * UNIQUE_PROB_RATIO:
        print(f"\nNOTE: Only {unique_probs}/{len(results)} unique probabilities")
        print("This suggests:")
        print("  - Model is using simple rules (e.g., 'appeared every year = 0.99')")
        print("  - Consider adding more features or increasing model complexity")
        print("  - For more granular predictions, try increasing n_estimators")

        # Show distribution of probabilities
        prob_counts = results['prob_2026'].value_counts().head(5)
        print(f"\n  Most common probabilities:")
        for prob, count in prob_counts.items():
            print(f"    {prob:.4f}: {count} topics")

    # Save everything
    val_df = save_results(results, all_results, next_year, INCLUDE_ALTERNATE, STRIDE)
    plot_validation_summary(all_results, best_n_lags, INCLUDE_ALTERNATE, STRIDE)

    print("\n" + "="*80)
    print("COMPLETE")
    print("="*80)


if __name__ == "__main__":
    main()

Loading data...
Data spans: 2015 to 2025 (11 years)
Topics tracked: 23

VALIDATION: Testing 9 configurations
Alternate lags: enabled (stride=2)

Testing 2-year lookback
  2020: Acc=0.870, F1=0.923 (trained on 5 years)
  2021: Acc=0.783, F1=0.872 (trained on 6 years)
  2022: Acc=0.826, F1=0.895 (trained on 7 years)
  2023: Acc=0.870, F1=0.923 (trained on 8 years)
  2024: Acc=0.783, F1=0.865 (trained on 9 years)
  2025: Acc=0.826, F1=0.900 (trained on 10 years)
  Average: Acc=0.826, F1=0.896 (±0.023)
  Range: F1 min=0.865, max=0.923
  Validated on 6 years

Testing 3-year lookback
  2021: Acc=0.826, F1=0.900 (trained on 6 years)
  2022: Acc=0.826, F1=0.895 (trained on 7 years)
  2023: Acc=0.870, F1=0.923 (trained on 8 years)
  2024: Acc=0.783, F1=0.872 (trained on 9 years)
  2025: Acc=0.870, F1=0.923 (trained on 10 years)
  Average: Acc=0.835, F1=0.903 (±0.019)
  Range: F1 min=0.872, max=0.923
  Validated on 5 years

Testing 4-year lookback
  2022: Acc=0.826, F1=0.895 (trained on 7 years)